In [15]:
import datetime
import BorealET_Input_Collection_v1 as bet_input
import BorealET_v1 as bet
import geemap
import ee

In [16]:
# Authenticate Google Earth Engine
ee.Authenticate()
ee.Initialize(project = 'borealet')

In [17]:
# Define region and period of interest. Here are some samples.
ROI = ee.Geometry.Polygon(
    [[[-148.4816422783826, 65.40125531420522],
      [-148.4816422783826, 64.18495817770238],
      [-145.3834977471326, 64.18495817770238],
      [-145.3834977471326, 65.40125531420522]]], None, False)
START_DATE = datetime.date(2001, 7, 1)
END_DATE = datetime.date(2001, 7, 8)

In [18]:
ROI = ee.Geometry.Polygon(
    [[[-148.4816, 65.4013],
      [-148.4816, 64.1849],
      [-145.3835, 64.1849],
      [-145.3835, 65.4013]]], None, False)

In [19]:
# Helper Functions

# Function to Calculate ET with BorealET
def et_fun(image):
    return bet.evapotranspiration(image).set('system:time_start', image.get('system:time_start'))

# Rescale images and add components to get total daily et (mm/day)
def scale_and_total_bet(image):
    # Divide by scale factor
    scaled = image.select(['e_canopy_day', 'e_soil_day', 't_day', 'e_canopy_night', 'e_soil_night', 't_night']).divide(1e7)
    
    # Add bands for daytime ET, nighttime ET, and total daily ET
    et_day = scaled.select('e_canopy_day').add(scaled.select('e_soil_day')).add(scaled.select('t_day')).rename('et_day')
    et_night = scaled.select('e_canopy_night').add(scaled.select('e_soil_night')).add(scaled.select('t_night')).rename('et_night')
    et_total = et_day.add(et_night).rename('et_total')
    
    # Add the new bands and copy properties from the original image
    return scaled.addBands([et_day, et_night, et_total]).set('system:time_start', image.get('system:time_start'))

# Clip images to ROI
def roi_clip(image):
    return image.clip(ROI)

# Visualization Parameters
bet_viz = {
    'min': 0,
    'max': 3.5,
    'palette': [
        'ffffff',  # white
        'c5e7f4',  # light blue
        '1532bf'   # dark blue
    ]
}

In [20]:
# Run BorealET
all_inputs = bet_input.aggregate_all_inputs(START_DATE, END_DATE)
bet_collection = (all_inputs
                  .map(et_fun)
                  .map(scale_and_total_bet)
                  .map(roi_clip))

In [21]:
# Print image collection (requires geemap)
bet_collection

In [22]:
# Add a sample image to the map. It might take a second to draw.
bet_image = bet_collection.first().select('et_total')
img_label = 'BET ' + str(START_DATE)

Map = geemap.Map(center=(64.8, -146.96), zoom = 8)
Map.addLayer(bet_image, bet_viz, img_label)
Map.addLayer(ROI, {}, 'ROI', False)
Map

Map(center=[64.8, -146.96], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…